# pandas高级应用

In [2]:
import numpy as np
import pandas as pd
%matplotlib inline

## 12.1分类数据

### 背景和目的
表中的一列通常会有重复的包含不同值的小集合的情况。我们已经学过了unique和value_counts，它们可以从数组提取出不同的值，并分别计算频率：

In [8]:
values = pd.Series(['apple','orange','apple']*2)
print(values)
print(pd.unique(values))
pd.value_counts(values)

0     apple
1    orange
2     apple
3     apple
4    orange
5     apple
dtype: object
['apple' 'orange']


apple     4
orange    2
dtype: int64

许多数据系统（数据仓库、统计计算或其它应用）都发展出了特定的表征重复值的方法，以进行高效的存储和计算。在数据仓库中，最好的方法是使用所谓的包含不同值的维表(Dimension Table)，将主要的参数存储为引用维表整数键：

In [11]:
values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple','orange'])
# 可以使用take方法存储原始的字符串Series：
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

这种用整数表示的方法称为分类或字典编码表示法。不同值得数组称为分类、字典或数据级。本书中，我们使用分类的说法。表示分类的整数值称为分类编码或简单地称为编码。
分类表示可以在进行分析时大大的提高性能。你也可以在保持编码不变的情况下，对分类进行转换。一些相对简单的转变例子包括：
- 重命名分类。
- 加入一个新的分类，不改变已经存在的分类的顺序或位置。

### pandas的分类类型
pandas有一个特殊的分类类型，用于保存使用整数分类表示法的数据。看一个之前的Series例子：

In [40]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
df = pd.DataFrame({'fruit':fruits,
                 'basket_id':np.arange(N),
                 'count':np.random.randint(3, 15, size=N),
                 'weight':np.random.uniform(0, 4, size=N)},
                  columns=['basket_id','fruit','count','weight'])
print("fruit_cat列的类型：\n",df.fruit)
df

fruit_cat列的类型：
 0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: object


,basket_id,fruit,count,weight
0,0,apple,5,2.467592
1,1,orange,9,0.300316
2,2,apple,6,3.865667
3,3,apple,7,2.394227
4,4,apple,7,2.595016
5,5,orange,11,0.202761
6,6,apple,13,0.022968
7,7,apple,3,0.908010


这里，df['fruit']是一个Python字符串对象的数组。我们可以通过调用它，将他转变为分类：

In [31]:
fruit_cat = df['fruit'].astype('category')
print(fruit_cat)
c = fruit_cat.values
print("\n类型:",type(c)) #fruit_cat的值部署Numpy数组，而是一个pandas.Categorical实例
print("\n属性",c.categories,c.codes) # 分类对象有categories和codes属性

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

类型: <class 'pandas.core.arrays.categorical.Categorical'>

属性 Index(['apple', 'orange'], dtype='object') [0 1 0 0 0 1 0 0]


你可将DataFrame的列通过分配转换结果，转换为分类对象：

In [35]:
df['fruit'] = df['fruit'].astype('category')
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

也可以讲Python的数据结构转为Categorical

In [42]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

如果你已经从其它源获得了分类编码，你还可以使用from_codes构造器：

In [43]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

但是分类变换的顺序取决于输入数据的顺序，categories数组的顺序会不同。当使用from_codes或其他的构造器时，你可以指定分类一个有意义的顺序

In [48]:
ordered_cat = pd.Categorical.from_codes(codes, categories,ordered=True)
ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

输出[foo < bar < baz]指明‘foo’位于‘bar’的前面，以此类推。

无序的分类实例可以通过as_ordered排序：

In [49]:
my_cats_2.as_ordered()

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

不仅仅是字符串，分类数组可以包括任意不可变类型。

### 用分类进行计算
某些pandas组件，比如groupby函数，更适合分类。还有一些函数可以使用有序标志位。

来看一些随机的数值数据，使用pandas.qcut面元函数。它会返回pandas.Categorical。

In [53]:
np.random.seed(12345)
draws = np.random.randn(10)
print(draws)
bins = pd.qcut(draws, 4)
bins

[-0.20470766  0.47894334 -0.51943872 -0.5557303   1.96578057  1.39340583
  0.09290788  0.28174615  0.76902257  1.24643474]


[(-0.557, -0.13], (0.38, 1.127], (-0.557, -0.13], (-0.557, -0.13], (1.127, 1.966], (1.127, 1.966], (-0.13, 0.38], (-0.13, 0.38], (0.38, 1.127], (1.127, 1.966]]
Categories (4, interval[float64]): [(-0.557, -0.13] < (-0.13, 0.38] < (0.38, 1.127] < (1.127, 1.966]]

加上标签的面元分类不包含数据面元边界的信息，因此可以使用groupby提取一些汇总信息：

In [55]:
bins = pd.Series(bins, name='quartile')
results = (pd.Series(draws)
           .groupby(bins)
           .agg(['count','min','max'])
           .reset_index())
results

,quartile,count,min,max
0,"(-0.557, -0.13]",3,-0.555730,-0.204708
1,"(-0.13, 0.38]",2,0.092908,0.281746
2,"(0.38, 1.127]",2,0.478943,0.769023
3,"(1.127, 1.966]",3,1.246435,1.965781


In [58]:
results['quartile']

0    (-0.557, -0.13]
1      (-0.13, 0.38]
2      (0.38, 1.127]
3     (1.127, 1.966]
Name: quartile, dtype: category
Categories (4, interval[float64]): [(-0.557, -0.13] < (-0.13, 0.38] < (0.38, 1.127] < (1.127, 1.966]]

### 用分类提高性能

如果是在一个特定数据集上做大量分析，将其转换为分类可以极大地提高效率。dataframe列的分类使用的内存通常少得多。

In [59]:
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo','bar','baz','qux'] * (N // 4))

In [60]:
categories = labels.astype('category') # 转换为分类

In [61]:
print(labels.memory_usage())
print(categories.memory_usage())

80000080
10000272


转换为分类不是没有代价的，但这是一次性的代价

In [62]:
%time _ = labels.astype('category')

Wall time: 400 ms


GroupBy使用分类操作明显更快，是因为底层的算法使用整数编码数组，而不是字符串数组。


### 分类方法
包含分类数据的Series有一些特殊的方法，类似于Series.str字符串方法。它还提供了方便的分类和编码的使用方法。看下面的Series：

In [66]:
s = pd.Series(['a','b','c','d'] * 2)
print(s)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: object


0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

cat属性提供了分类方法的入口

In [70]:
print(cat_s.cat.codes,"\n")
print(cat_s.cat.categories)

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8 

Index(['a', 'b', 'c', 'd'], dtype='object')


假设我们知道这个数据的实际分类集，超出了数据中的四个值。我们可以使用set_categories方法改变它们：

In [71]:
actual_categories = ['a','b','c','d','e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

虽然数据看起来没变，新的分类将反映在它们的操作中。例如，如果有的话，value_counts表示分类：

In [74]:
print(cat_s.value_counts())
cat_s2.value_counts()

d    2
c    2
b    2
a    2
dtype: int64


d    2
c    2
b    2
a    2
e    0
dtype: int64

在大数据集中，分类经常作为节省内存和高性能的便捷工具。过滤完大DataFrame或Series之后，许多分类可能不会出现在数据中。我们可以使用remove_unused_categories方法删除没看到的分类：

In [83]:
cat_s3 = cat_s[cat_s.isin(['a','b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [81]:
cat_s.cat.remove_unused_categories()

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

Series的分类方法
![](https://i.loli.net/2019/08/21/qomk62BYIOdHZvp.jpg)

### 为建模创建虚拟变量
当你使用统计或机器学习工具时，通常会将分类数据转换为虚拟变量，也称为one-hot编码。这包括创建一个不同类别的列的DataFrame；这些列包含给定分类的1s，其它为0。

In [84]:
cat_s = pd.Series(['a','b','c','d'] * 2,dtype='category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [88]:
pd.get_dummies(cat_s) # 转换这个分类数据为包含虚拟变量的DataFrame

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## 12.2 GroupBy高级应用

### 分组转换和“解封”GroupBy
在第10章的分组操作中学习了可以使用apply方法，进行转换。还有另一个transform方法，它与apply很像，但是对使用的函数有一定限制：
- 它可以产生向分组形状广播标量值
- 它可以产生一个和输入形状相同的对象
- 它不能修改输入

In [89]:
df = pd.DataFrame({'key':['a','b','c'] * 4,
                   'value':np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


按key分组

In [94]:
g = df.groupby('key').value
g.min()

key
a    0.0
b    1.0
c    2.0
Name: value, dtype: float64

假设我们想产生一个和df['value']形状相同的Series，但值替换为按键分组的平均值。

In [95]:
g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

与apply类似，transform的函数会返回Series，但是结果必须与输入大小相同。举个例子，我们可以用lambda函数将每个分组乘以2：

In [96]:
g.transform(lambda x: x*2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

再举一个复杂的例子，我们可以计算每个分组的降序排名：

In [97]:
g.transform(lambda x: x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

看一个由简单句和构造的分组转换函数：

In [98]:
def normalize(x):
    return (x - x.mean()) / x.std()

我们用transform或apply可以获得等价的结果：

In [102]:
print(g.transform(normalize))
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64


0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

内置的聚合函数，比如mean和sum，通常比apply函数快，也比transform快。这允许我们进行一个所谓的解封(unwrapped)分组操作：

In [105]:
print(g.transform('mean'))
normalized = (df['value']-g.transform('mean')) / g.transform('std')
normalized

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64


0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

解封分组操作可能包括多个分组聚合，但是矢量化操作还是会带来收益。


### 分组的时间重采样
对于时间序列数据，resample方法从语义上是一个基于内在时间的分组操作。下面是一个示例表：

In [107]:
N = 15
times = pd.date_range('2019-05-20 00:00', freq='1min', periods=N)
df = pd.DataFrame({'time' : times,
                   'value': np.arange(N)})
df

,time,value
0,2019-05-20 00:00:00,0
1,2019-05-20 00:01:00,1
2,2019-05-20 00:02:00,2
3,2019-05-20 00:03:00,3
4,2019-05-20 00:04:00,4
5,2019-05-20 00:05:00,5
6,2019-05-20 00:06:00,6
7,2019-05-20 00:07:00,7
8,2019-05-20 00:08:00,8
9,2019-05-20 00:09:00,9


这里我们可以用time作为索引，然后重采样：

In [108]:
df.set_index('time').resample('5min').count()

,value
time,
2019-05-20 00:00:00,5
2019-05-20 00:05:00,5
2019-05-20 00:10:00,5


假设DataFrame包含多个时间序列，用一个额外的分组键的列进行标记：

In [111]:
df2 = pd.DataFrame({'time':times.repeat(3),
                    'key':np.tile(['a','b','c'], N),
                    'value':np.arange(N * 3.)})
df2[:7]

,time,key,value
0,2019-05-20 00:00:00,a,0.0
1,2019-05-20 00:00:00,b,1.0
2,2019-05-20 00:00:00,c,2.0
3,2019-05-20 00:01:00,a,3.0
4,2019-05-20 00:01:00,b,4.0
5,2019-05-20 00:01:00,c,5.0
6,2019-05-20 00:02:00,a,6.0


要对每个key值进行重新采样，我们引入pandas.TimeGrouper对象：

In [117]:
time_key = pd.TimeGrouper('5min')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  """Entry point for launching an IPython kernel.


设定时间索引，用key和time_key分组，然后聚合：

In [118]:
resampled = (df2.set_index('time')
            .groupby(['key', time_key])
             .sum())
resampled

value
key time                      
a   2019-05-20 00:00:00   30.0
    2019-05-20 00:05:00  105.0
    2019-05-20 00:10:00  180.0
b   2019-05-20 00:00:00   35.0
    2019-05-20 00:05:00  110.0
    2019-05-20 00:10:00  185.0
c   2019-05-20 00:00:00   40.0
    2019-05-20 00:05:00  115.0
    2019-05-20 00:10:00  190.0

In [119]:
resampled.reset_index()

,key,time,value
0,a,2019-05-20 00:00:00,30.0
1,a,2019-05-20 00:05:00,105.0
2,a,2019-05-20 00:10:00,180.0
3,b,2019-05-20 00:00:00,35.0
4,b,2019-05-20 00:05:00,110.0
5,b,2019-05-20 00:10:00,185.0
6,c,2019-05-20 00:00:00,40.0
7,c,2019-05-20 00:05:00,115.0
8,c,2019-05-20 00:10:00,190.0


使用TimeGrouper的限制是时间必须是Series或DataFrame索引。